# Парсинг заголовков сайтов компаний по справочнику Санкт-Петербурга

In [1]:
import pandas as pd
import requests

In [2]:
df=pd.read_excel('база предприятий спб/Санкт-Петербург 2ГИС.xlsx')  # Это то, куда вы скачали файл
df.info()

In [3]:
# Удаление строк в которых отсутствует адрес сайта
df.dropna(subset=['Сайт'], inplace=True)

In [4]:
# Удаление лишних столбцов
df.drop(df.columns[range(8,46)], axis=1, inplace=True)

In [3]:
df.info()
df.head()

In [6]:
df.describe(include='all')

In [7]:
# уникальных адресов сайтов 40317 шт, удалит дубликаты
df=df.drop_duplicates('Сайт')
df.info()
df.head()

In [15]:
Сбросить индекс строк
df=df.reset_index(drop=True)
делать не обязательно, так как новый скрипт сам определяет индексы.

In [5]:
df.head(10)

## Парсинг актуального адреса сайта с сохранением истории редиректов.

In [7]:

# Данный способ лучше, так как не нужно каждый раз определять место на котором прервался и с которого нужно начинать.

# Второй и последующие запуски

df=pd.read_pickle('https.pkl')

from tqdm import tnrange, tqdm_notebook, tqdm
from time import sleep
import threading


#определяем индексы пустых ячеек
index = df[df['url'].isna()].index

#количество потоков
n=10

#создание функции 
def urlverfctn(i):
    """
    i - номер строки в датафреме
    """
    try:
        url=df.loc[i,'Сайт']
        r = requests.head(url, timeout=(30, 200), allow_redirects=True)
        df.loc[i,'url']= r.url
        history=''
        if len(r.history)>0:
            for h in range(len(r.history)):
                history=history + r.history[h].url+"-"+str(r.history[h].status_code)+'->'
            history=history+r.url
        df.loc[i,'history']= history
    except Exception as e:
        df.loc[i,'url']= ("Error! " + str(e))
        print("Error! " + str(e))
    print('OK', i, history)

    
# Настройка потоков  
for i in tnrange(0, len(index), n):
    #массив для добавления потоков
    threads = []
    # переменная для задания диапазона строк
    m=i+n
    for j in range(i,m):
        # Подготавливаем потоки, складываем их в массив
        threads.append(threading.Thread(target=urlverfctn, args=(index[j],)))
    # Запускаем каждый поток
    for thread in threads:
        thread.start()
    # Завершение каждого потока
    for thread in threads:
        thread.join()
    df.to_pickle('https.pkl')

In [8]:
# Проверка адресов, которые выдали ошибку.
# Парсинг актуального адреса сайта с сохранением истории редиректов.

df=pd.read_pickle('https.pkl')

from tqdm import tnrange, tqdm_notebook, tqdm
from time import sleep
import threading
import logging
import warnings
warnings.filterwarnings('ignore')

logging.basicConfig(format = u'%(levelname)-8s [%(asctime)s] %(message)s', level = logging.I, filename = u'mylog.log')


#определяем индексы ячеек с ошибками 
index = df[df['url'].str.find('Error!')>-1][5100:].index

#количество потоков
n=10

#создание функции 
def urlverfctn(i):
    """
    i - номер строки в датафреме
    """
    try:
        url=df.loc[i,'Сайт']
        r = requests.head(url, timeout=(30, 200), allow_redirects=True)
        df.loc[i,'url']= r.url
        history=''
        if len(r.history)>0:
            for h in range(len(r.history)):
                history=history + r.history[h].url+"-"+str(r.history[h].status_code)+'->'
            history=history+r.url
        df.loc[i,'history']= history
    except Exception as e:
        df.loc[i,'url']= ("Error! " + str(e))
        logging.error("Error! " + str(e), i)
    logging.info('OK', i, history)

    
# Настройка потоков  
for i in tnrange(0, len(index), n):
    #массив для добавления потоков
    threads = []
    # переменная для задания диапазона строк
    m=i+n
    for j in range(i,m):
        # Подготавливаем потоки, складываем их в массив
        threads.append(threading.Thread(target=urlverfctn, args=(index[j],)))
    # Запускаем каждый поток
    for thread in threads:
        thread.start()
    # Завершение каждого потока
    for thread in threads:
        thread.join()
    df.to_pickle('https.pkl')

In [2]:
df=pd.read_pickle('https_clean.pkl')

In [12]:
# df1=df[df['url'].str.find('Error!')>-1].reset_index()

In [10]:
# df1[df1['url'].str.find('www.bsc-ltd.ru')>-1]

In [11]:
# df[df['url'].str.find('Error!')>-1][5100:]


In [20]:
# df[df['url'].str.find('Error!')>-1]

In [21]:
# df[df['url'].str.find('Error!')>-1].index

In [26]:
# df.to_pickle('https.pkl')

In [ ]:
# df['https']=df['Сайт'].str.find('https')>-1

In [12]:
df['https']=df['url'].str.find('https')>-1

In [13]:
df['https'].value_counts()


False    32672
True      7645
Name: https, dtype: int64

## Вывод:
### Только 18,96% компаний используют https соединение.

In [17]:
df=df.drop(df[df['Рубрика']=='Город / Власть'].index)
df.head()

,Город,Наименование,Адрес,Рубрика,Подрубрика,Сайт,Почта,Тел. 1,url,history,https
0,Санкт-Петербург,"Аварийно-диспетчерская служба Невского района,...","Дыбенко, 25 к5",Аварийные / справочные / экстренные службы,Аварийные службы,http://www.gks1nev.ru,NaN,(812) 308-88-01,https://www.gks1nev.ru/,http://www.gks1nev.ru/-301->https://www.gks1ne...,True
1,Санкт-Петербург,"Центрально-диспетчерская служба, ООО Петербург...","Разъезжая, 46",Аварийные / справочные / экстренные службы,Аварийные службы,http://www.ptenergo.ru,office@ptenergo.ru,(812) 233-00-08,https://ptenergo.ru,http://www.ptenergo.ru/-302->https://ptenergo.ru,True
2,Санкт-Петербург,"Аварийно-диспетчерская служба Выборгского, Кал...","Кондратьевский проспект, 15 к3",Аварийные / справочные / экстренные службы,Аварийные службы,http://cityservice.spb.ru,cityservice@spcs.spb.ru,(812) 604-00-00,https://cityservice.spb.ru/,http://cityservice.spb.ru/-302->https://cityse...,True
3,Санкт-Петербург,"Аварийка.ру, аварийная служба","Набережная реки Фонтанки, 90 к5",Аварийные / справочные / экстренные службы,Аварийные службы,http://www.avariyca.ru,avariyca@yandex.ru,(812) 926-96-39,http://www.avariyca.ru/,,False
4,Санкт-Петербург,"Прогресс, аварийно-восстановительная служба","Костромской проспект, 4",Аварийные / справочные / экстренные службы,Аварийные службы,http://guprep.ru,progress-plan@rambler.ru,(812) 554-48-18,http://guprep.ru/,,False


In [4]:
df['Рубрика'].unique()

array(['Аварийные / справочные / экстренные службы',
       'Автосервис / Автотовары',
       'Досуг / Развлечения / Общественное питание',
       'Интернет / Связь / Информационные технологии',
       'Коммунальные / бытовые / ритуальные услуги',
       'Компьютеры / Бытовая техника / Офисная техника',
       'Культура / Искусство / Религия', 'Мебель / Материалы / Фурнитура',
       'Медицина / Здоровье / Красота', 'Металлы / Топливо / Химия',
       'Оборудование / Инструмент', 'Образование / Работа / Карьера',
       'Одежда / Обувь', 'Охрана / Безопасность',
       'Продукты питания / Напитки', 'Реклама / Полиграфия / СМИ',
       'Спорт / Отдых / Туризм', 'Строительные / отделочные материалы',
       'Строительство / Недвижимость / Ремонт',
       'Текстиль / Предметы интерьера',
       'Товары для животных / Ветеринария',
       'Торговые комплексы / Спецмагазины', 'Транспорт / Грузоперевозки',
       'Хозтовары / Канцелярия / Упаковка',
       'Электроника / Электротехника',
   

In [9]:
df[df['Подрубрика']=='Аварийные службы']

,Город,Наименование,Адрес,Рубрика,Подрубрика,Сайт,Почта,Тел. 1,url,history,https
0,Санкт-Петербург,"Аварийно-диспетчерская служба Невского района,...","Дыбенко, 25 к5",Аварийные / справочные / экстренные службы,Аварийные службы,http://www.gks1nev.ru,NaN,(812) 308-88-01,https://www.gks1nev.ru/,http://www.gks1nev.ru/-301->https://www.gks1ne...,True
1,Санкт-Петербург,"Центрально-диспетчерская служба, ООО Петербург...","Разъезжая, 46",Аварийные / справочные / экстренные службы,Аварийные службы,http://www.ptenergo.ru,office@ptenergo.ru,(812) 233-00-08,https://ptenergo.ru,http://www.ptenergo.ru/-302->https://ptenergo.ru,True
2,Санкт-Петербург,"Аварийно-диспетчерская служба Выборгского, Кал...","Кондратьевский проспект, 15 к3",Аварийные / справочные / экстренные службы,Аварийные службы,http://cityservice.spb.ru,cityservice@spcs.spb.ru,(812) 604-00-00,https://cityservice.spb.ru/,http://cityservice.spb.ru/-302->https://cityse...,True
3,Санкт-Петербург,"Аварийка.ру, аварийная служба","Набережная реки Фонтанки, 90 к5",Аварийные / справочные / экстренные службы,Аварийные службы,http://www.avariyca.ru,avariyca@yandex.ru,(812) 926-96-39,http://www.avariyca.ru/,,False
4,Санкт-Петербург,"Прогресс, аварийно-восстановительная служба","Костромской проспект, 4",Аварийные / справочные / экстренные службы,Аварийные службы,http://guprep.ru,progress-plan@rambler.ru,(812) 554-48-18,http://guprep.ru/,,False


In [26]:
df[df['Рубрика']=='Интернет / Связь / Информационные технологии'].head()

,Город,Наименование,Адрес,Рубрика,Подрубрика,Сайт,Почта,Тел. 1,url,history,https
5095,Санкт-Петербург,"Дом.ru Бизнес, телекоммуникационный центр, фил...","Коломяжский проспект, 27",Интернет / Связь / Информационные технологии,IP-телефония,http://www.b2b.domru.ru,b2b@ertelecom.ru,8-800-333-90-00,https://interzet.domru.ru/,http://www.b2b.domru.ru/-301->http://domru.ru-...,True
5096,Санкт-Петербург,"ЭлектронТелеком, телекоммуникационная компания","Коломяжский проспект, 18",Интернет / Связь / Информационные технологии,IP-телефония,http://www.etelecom.ru,info@etelecom.ru;et@etelecom.ru,(812) 602-26-26,http://www.etelecom.ru/,,False
5097,Санкт-Петербург,"MNS, ООО Матрикс Интернет Клуб","Комендантский проспект, 28 к2",Интернет / Связь / Информационные технологии,IP-телефония,http://www.mns.ru,sales@mns.ru,(812) 438-46-00,http://www.mns.ru/,,False
5098,Санкт-Петербург,"КОМСТАР-Регионы, телекоммуникационная компания","Просвещения проспект, 85",Интернет / Связь / Информационные технологии,IP-телефония,http://www.dom.mts.ru,oplata@spb.mts.ru,8-800-250-08-90,https://spb.mts.ru:443/personal/dom/,http://www.dom.mts.ru/-302->http://www.mts.ru/...,True
5099,Санкт-Петербург,"Петербургская Интернет Сеть, телекоммуникацион...","Обуховской Обороны проспект, 120 лит Б",Интернет / Связь / Информационные технологии,IP-телефония,http://www.pinspb.ru,info@pinspb.ru,(812) 677-25-25,https://pinspb.ru//,http://www.pinspb.ru/-301->https://pinspb.ru//,True


In [43]:
df.describe()

,Город,Наименование,Адрес,Рубрика,Подрубрика,Сайт,Почта,Тел. 1,url,history,https
count,39537,39537,38626,39537,39537,39537,32919,32899,39535,32208,39537
unique,1,39392,13317,26,949,39537,32693,32445,38672,13461,2
top,Санкт-Петербург,"СПбГУ, Санкт-Петербургский государственный уни...","Варшавская, 3 к1",Строительство / Недвижимость / Ремонт,Туристические агентства,http://www.autoralf.ru,cardiomedika@inbox.ru,(812) 328-20-00,"Error! HTTPSConnectionPool(host='faitid.org', ...",,False
freq,39537,15,125,3201,867,1,6,15,411,18748,32036
